In [1]:
import pandas as pd
from PyPDF2 import PdfReader, PdfWriter
from reportlab.pdfgen import canvas
from reportlab.lib.pagesizes import letter
from reportlab.lib.colors import Color
from reportlab.pdfbase import pdfmetrics
from reportlab.pdfbase.ttfonts import TTFont

pdfmetrics.registerFont(TTFont("NotoSansGujarati", "NotoSansGujarati-Medium.ttf"))

def create_temp_pdf(name, x, y, output_path):
    """Creates a temporary PDF with the name at specified coordinates."""
    c = canvas.Canvas(output_path, pagesize=letter)
    c.setFont("NotoSansGujarati", 16)
    custom_color = Color(250/255, 128/255, 114/255) # RGB value format, where it takes range from 0 to 1 only.
    c.setFillColor(custom_color)
    c.drawString(x, y, name.encode('utf-8').decode('utf-8'))  # Set position (x, y) for the name
    c.save()

def add_name_to_pages(template_path, output_path, name, positions):
    """
    Adds the given name to the specified pages of the template PDF.
    positions: A dictionary where keys are page indices (0-based) and values are (x, y) coordinates.
    """
    writer = PdfWriter()
    template_pdf = PdfReader(template_path)

    for i, page in enumerate(template_pdf.pages):
        if i in positions:  # Check if the current page is in the target positions
            x, y = positions[i]
            temp_pdf_path = "temp_page.pdf"
            create_temp_pdf(name, x, y, temp_pdf_path)  # Create temporary PDF for this page
            temp_pdf = PdfReader(temp_pdf_path)
            page.merge_page(temp_pdf.pages[0])  # Merge temporary PDF onto the current page
        writer.add_page(page)  # Add the (modified or unmodified) page to the output

    # Save the customized PDF
    with open(output_path, "wb") as output_file:
        writer.write(output_file)

def process_invites(template_path, excel_path, output_dir):
    """Processes the Excel file and generates PDFs for all names."""
    # Read the Excel file
    df = pd.read_excel(excel_path)

    # Define target pages and positions
    # Format: {page_index: (x, y)}
    positions = {
        0: (170, 300),  # Page 1
        2: (110, 528),  # Page 3
    }

    # Loop through each name in the Excel file
    for index, row in df.iterrows():
        name = row["Name"]  # Adjust the column name as per your Excel
        mobnum = row["Number"]
        name_guj = row["Name_Guj"]  # Adjust the column name as per your Excel
        output_file = f"{output_dir}/Vaniya Family Invitation_{mobnum}_{name_guj}.pdf"
        add_name_to_pages(template_path, output_file, name, positions)
        print(f"Generated: {output_file}")

# Paths to your files
template_pdf = "template.pdf"
excel_file = "names.xlsx"
output_directory = "output_invites"

# Process the invitations
process_invites(template_pdf, excel_file, output_directory)

Generated: output_invites/Vaniya Family Invitation_9712424630_શ્રુતિ લલિતકુમાર વાણીયા.pdf
Generated: output_invites/Vaniya Family Invitation_9409435179_શિવાની નરેન્દ્રભાઈ કુવાડ.pdf


In [2]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# Folder Path containing the PDF files
PDF_FOLDER = "/Users/ravivaniya/Developer/mari-kankotri/output_invites"
INVITE_MESSAGE = " 🙏 નમસ્કાર, અમારા લગ્ન પ્રસંગે આપ સહપરિવાર આવીને અમને આશીર્વાદ આપશો, આપની હાજરી અમારી માટે અમૂલ્ય રહેશે. 💐"

# Delay settings
WAIT_TIME = 20

# Function to initialize the WebDriver
def init_driver():
    options = webdriver.ChromeOptions()
    options.add_argument("--user-data-dir=./whatsapp_profile")  # Save WhatsApp session
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=options)
    return driver

# Function to send a message and attach a file
def send_message(driver, mobile_number, user_name, file_path):
    try:
        # Open WhatsApp Web chat
        url = f"https://web.whatsapp.com/send?phone={mobile_number}&text=જય%20સ્વામિનારાયણ%0A%2A{user_name}!%2A%0A{INVITE_MESSAGE}"
        driver.get(url)
        time.sleep(50)  # Wait for the chat to load

        # Locate the attachment button
        attach_button = driver.find_element(By.XPATH, "//span[@data-icon='plus']")
        attach_button.click()
        time.sleep(10)

        # Upload the PDF file
        file_input = driver.find_element(By.XPATH, "//input[@accept='*']")
        file_input.send_keys(file_path)
        time.sleep(50)  # Wait for the file to upload

        # Click the send button
        send_button = driver.find_element(By.XPATH, "//span[@data-icon='send']")
        send_button.click()
        time.sleep(10)  # Wait for the message to be sent

        print(f"Message sent to {mobile_number} {user_name}")
    except Exception as e:
        print(f"Failed to send message to {mobile_number}: {e}")

# Main Logic
if __name__ == "__main__":
    # Initialize WebDriver
    driver = init_driver()

    # Wait for the user to scan the QR code
    print("Please scan the QR code to log in to WhatsApp Web.")
    driver.get("https://web.whatsapp.com/")
    time.sleep(60)

    for file_name in os.listdir(PDF_FOLDER):
        if file_name.endswith(".pdf"):
            # Extract mobile number and user name from file name
            parts = file_name.replace(".pdf", "").split("_")
            if len(parts) == 3:
                temp_msg, mobile_number, user_name = parts
                pdf_path = os.path.join(PDF_FOLDER, file_name)

                # Send the message and PDF
                send_message(driver, mobile_number, user_name, pdf_path)

                # Wait before processing the next user
                time.sleep(WAIT_TIME)
            else:
                print(f"Invalid file name format: {file_name}")

    # Close the WebDriver
    driver.quit()


Please scan the QR code to log in to WhatsApp Web.
Message sent to 9712424630 શ્રુતિ લલિતકુમાર વાણીયા
Message sent to 9409435179 શિવાની નરેન્દ્રભાઈ કુવાડ
